In [ ]:
# =============================================================================
# Практикум 2026: BERTopic — тематическое моделирование смешанного корпуса
# Сравнение разных эмбеддингов + визуализация (datamapplot, wordcloud)
# =============================================================================

# Установка (один раз)
# !pip install -q bertopic datasets sentence-transformers umap-learn hdbscan datamapplot wordcloud pymorphy3 nltk tqdm seaborn

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from tqdm.auto import tqdm

from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance
from umap import UMAP
from hdbscan import HDBSCAN
import datamapplot

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)
ru_stop = set(stopwords.words('russian'))

import pymorphy3
morph = pymorphy3.MorphAnalyzer()

%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')

print("Библиотеки загружены ✓\n")

# ─── 1. Сбор и подготовка корпуса ───────────────────────────────────────────

print("1. Загружаем ArXiv NLP + русские тексты 2026 года...")

# ArXiv (английские абстракты по NLP)
ds_arxiv = load_dataset("maartengr/arxiv_nlp", split="train")
arxiv_abs = ds_arxiv["Abstracts"][:450]  # берём 450 для скорости

# Русские тексты (актуальные темы 2026)
ru_docs = [
    "В 2026 году AGI впервые достигнут одновременно в xAI, Anthropic и DeepMind. Контекстное окно превысило 5 млн токенов.",
    "Grok 4 пишет научные статьи уровня Nature за 2 минуты по текстовому описанию.",
    "Термоядерный синтез SPARC достиг Q=15. Первая коммерческая станция — 2034 год.",
    "CRISPR-Cas15 редактирует геном взрослых без off-target эффектов.",
    "Квантовые чипы с 8000 кубитами решают задачи оптимизации мгновенно.",
    "Глобальное потепление +2.4°C. Уровень моря поднялся на 22 см за 18 лет.",
    "Starship Block 5 выполнил 18 орбитальных полётов с возвращением обеих ступеней."
]

all_texts = list(arxiv_abs) + ru_docs
print(f"Всего документов: {len(all_texts):,} (ArXiv + русский)\n")

# ─── 2. Предобработка (лемматизация + биграммы) ─────────────────────────────

def clean_text_ru(text: str) -> list[str]:
    text = text.lower()
    tokens = [w for w in text.split() if w.isalpha() and w not in ru_stop]
    lemmas = [morph.parse(t)[0].normal_form for t in tokens if len(t) >= 3]
    return lemmas

print("Предобработка...")
tokenized = [clean_text_ru(doc) for doc in tqdm(all_texts)]

# Биграммы
bigram = Phrases(tokenized, min_count=3, threshold=7)
bigram_ph = Phraser(bigram)
tokenized = [bigram_ph[doc] for doc in tokenized]

print("Пример обработанного текста:")
print("  " + " ".join(tokenized[0][:30]) + "...\n")

# ─── 3. Сравнение эмбеддингов ───────────────────────────────────────────────

print("3. Сравниваем эмбеддинг-модели для BERTopic\n")

embedding_variants = {
    "multilingual-e5-large-instruct": "intfloat/multilingual-e5-large-instruct",
    "LaBSE": "sentence-transformers/LaBSE",
    "paraphrase-multilingual-mpnet-base-v2": "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
}

topic_models = {}
summary_stats = []

for emb_name, emb_path in tqdm(embedding_variants.items(), desc="Эмбеддинги"):
    print(f"\n→ Обработка с {emb_name}")
    
    embedder = SentenceTransformer(emb_path)
    
    umap = UMAP(
        n_neighbors=12,
        n_components=5,
        metric='cosine',
        min_dist=0.08,
        random_state=2026
    )
    
    hdb = HDBSCAN(
        min_cluster_size=35,
        min_samples=8,
        metric='euclidean',
        cluster_selection_method='leaf'
    )
    
    representation = {
        "KeyBERT": KeyBERTInspired(),
        "MMR": MaximalMarginalRelevance(diversity=0.38)
    }
    
    topic_model = BERTopic(
        embedding_model=embedder,
        umap_model=umap,
        hdbscan_model=hdb,
        representation_model=representation,
        verbose=True,
        calculate_probabilities=True,
        nr_topics="auto"
    )
    
    topics, probs = topic_model.fit_transform(all_texts)
    
    n_themes = len(set(topics)) - (1 if -1 in topics else 0)
    noise = sum(t == -1 for t in topics) / len(topics) * 100
    
    topic_models[emb_name] = topic_model
    summary_stats.append({
        "Эмбеддинг": emb_name,
        "Тем": n_themes,
        "Шум (%)": round(noise, 1),
        "Документов в темах": len(topics) - sum(t == -1 for t in topics)
    })
    
    print(f"   Тем: {n_themes} | Шум: {noise:.1f}%")

# ─── 4. Сравнительная таблица эмбеддингов ───────────────────────────────────

print("\nСравнение эмбеддингов:")
stats_df = pd.DataFrame(summary_stats).sort_values("Тем", ascending=False)
display(stats_df.style.format(precision=1).background_gradient(cmap='YlGnBu'))

best_emb = stats_df.iloc[0]["Эмбеддинг"]
print(f"\nЛучший результат: {best_emb} (максимум тем и минимум шума)\n")

# ─── 5. Полная визуализация лучшей модели ───────────────────────────────────

best_model = topic_models[best_emb]

print(f"Интерактивная карта тем ({best_emb})")
reduced = best_model.umap_model.embedding_[:, :2]

fig = datamapplot.create_interactive_map(
    reduced,
    best_model.topics_,
    best_model.get_topic_info()["Name"],
    hover_text=[f"Тема {t}\n{doc[:160]}..." for t, doc in zip(best_model.topics_, all_texts)],
    label_fontsize=10,
    point_size=6,
    title=f"Тематическая карта ArXiv NLP + русские тексты 2026 ({best_emb})"
)
fig.show()

# ─── 6. WordCloud для топ-тем ───────────────────────────────────────────────

print("\nОблака слов для топ-8 тем")
for tid in range(8):
    if tid in best_model.topic_representations_:
        terms = best_model.get_topic(tid)
        if terms:
            freq = {w: p for w, p in terms[:70]}
            wc = WordCloud(width=700, height=450, background_color='white',
                          colormap='inferno', max_words=50).generate_from_frequencies(freq)
            plt.figure(figsize=(9,6))
            plt.imshow(wc, interpolation='bilinear')
            plt.axis("off")
            plt.title(f"Тема {tid}: {best_model.get_topic_info().loc[tid, 'Name'][:70]}")
            plt.show()

# ─── 7. Топ-документы по темам ──────────────────────────────────────────────

print("\nТоп-5 документов по каждой теме")
topic_df = pd.DataFrame({
    "text": all_texts,
    "topic": best_model.topics_,
    "prob": best_model.probabilities_.max(axis=1)
})

for tid in sorted(topic_df["topic"].unique()):
    if tid == -1:
        continue
    top_docs = topic_df[topic_df["topic"] == tid].nlargest(5, "prob")
    theme_name = best_model.get_topic_info().loc[tid, 'Name'][:80]
    print(f"\nТема {tid}: {theme_name}")
    for _, row in top_docs.iterrows():
        print(f"  {row['prob']:.3f} → {row['text'][:140]}...")
    print("─"*90)

print("\nПрактикум завершён. Можно менять эмбеддинги, параметры HDBSCAN/UMAP и корпус!")